In [ ]:
import json
from os import replace
!which python

In [ ]:
!pip install pyarrow

In [2]:
import pandas as pd
import requests
import json
import pprint


In [16]:
agreementUuid = "ecd33086-3972-43e3-aa49-93bc7cdbd04a"
agreementPid = "urn:uuid:" + agreementUuid

# Testing HL Api for PULL transfer

---
## Setup transfer

In [17]:
url = "http://localhost:1235/api/v1/setup-transfer"

payload = ""
headers = {}
response = requests.request("POST", url, headers=headers, data=payload)
callbackAddress = response.json()["callbackAddress"]
callbackId = response.json()["callbackId"]
consumerPid = response.json()["consumerPid"]


pprint.pprint(response.json())

{'callbackAddress': 'http://localhost:1235/3f970a41-fde1-4a7e-afbb-c5d58d3a7499/transfers/67500ef8-530a-4269-bf41-79c9f918cf7f',
 'callbackId': '3f970a41-fde1-4a7e-afbb-c5d58d3a7499',
 'consumerPid': 'urn:uuid:67500ef8-530a-4269-bf41-79c9f918cf7f'}


## Request transfer

In [18]:
url = "http://localhost:1235/api/v1/request-transfer"

payload = json.dumps({
  "agreementId": agreementPid,
  "format": "http+pull",
  "dataAddress": None,
  "callbackAddress": callbackAddress,
  "callbackId": callbackId,
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())


{'consumerPid': 'urn:uuid:67500ef8-530a-4269-bf41-79c9f918cf7f',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:67500ef8-530a-4269-bf41-79c9f918cf7f',
                     'dspace:providerPid': 'urn:uuid:a56de3eb-8c9a-404b-a254-e59c197f49ae',
                     'dspace:state': 'dspace:REQUESTED'}}


## Get data address

In [19]:
import requests

consumerPidUuid = consumerPid.replace("urn:uuid:", "")
url = "http://localhost:1235/api/v1/data-address/" + consumerPidUuid

payload = ""
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)
dataPlaneAddress = response.json()["dataPlaneAddress"]

pprint.pprint(response.json())

{'dataPlaneAddress': 'http://localhost:1235/3f970a41-fde1-4a7e-afbb-c5d58d3a7499/data/67500ef8-530a-4269-bf41-79c9f918cf7f'}


## Use data address to get data from data space

In [20]:
url = dataPlaneAddress

df = pd.read_parquet(url)
df

,column0,column1
0,first,last
1,Jorge,Frank
2,Hunter,Moreno
3,Esther,Guzman
4,Dennis,Stephens
...,...,...
96,Brett,Miller
97,Adelaide,Phelps
98,Leo,Walters
99,Samuel,Schmidt


## Suspend transfer

In [9]:
url = "http://localhost:1235/api/v1/suspend-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:5a4a01f6-f28b-480f-babc-8907595c9b4f',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:5a4a01f6-f28b-480f-babc-8907595c9b4f',
                     'dspace:providerPid': 'urn:uuid:8670c0c4-e0a9-425e-a2a9-b995180d0bd4',
                     'dspace:state': 'dspace:SUSPENDED'}}


## Test data access has been suspended

In [10]:
url = dataPlaneAddress

df2 = pd.read_parquet(url)
df2

HTTPError: HTTP Error 401: Unauthorized

## Restart transfer

In [11]:
url = "http://localhost:1235/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:5a4a01f6-f28b-480f-babc-8907595c9b4f',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:5a4a01f6-f28b-480f-babc-8907595c9b4f',
                     'dspace:providerPid': 'urn:uuid:8670c0c4-e0a9-425e-a2a9-b995180d0bd4',
                     'dspace:state': 'dspace:STARTED'}}


## Test data is accessible again

In [12]:
url = dataPlaneAddress

df3 = pd.read_parquet(url)
df3

,column0,column1
0,first,last
1,Jorge,Frank
2,Hunter,Moreno
3,Esther,Guzman
4,Dennis,Stephens
...,...,...
96,Brett,Miller
97,Adelaide,Phelps
98,Leo,Walters
99,Samuel,Schmidt


## Complete transfer

In [13]:
url = "http://localhost:1235/api/v1/complete-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:5a4a01f6-f28b-480f-babc-8907595c9b4f',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:5a4a01f6-f28b-480f-babc-8907595c9b4f',
                     'dspace:providerPid': 'urn:uuid:8670c0c4-e0a9-425e-a2a9-b995180d0bd4',
                     'dspace:state': 'dspace:COMPLETED'}}


## Data is not accessible

In [14]:
url = dataPlaneAddress

df3 = pd.read_parquet(url)
df3

HTTPError: HTTP Error 401: Unauthorized

## Protocol state cannot change

In [15]:
url = "http://localhost:1235/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:5a4a01f6-f28b-480f-babc-8907595c9b4f',
 'error': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
           '@type': 'dspace:TransferError',
           'dspace:code': 'TRANSFER_ERROR_CODE',
           'dspace:consumerPid': '123',
           'dspace:providerPid': '123',
           'dspace:reason': ['Protocol Error. dspace:TransferRequestMessage is '
                             'not allowed here. Current state is '
                             'dspace:COMPLETED']}}
